In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vuppalaadithyasairam/bone-fracture-detection-using-xrays")

print("Path to dataset files:", path)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# used to find all the pathnames matching
import glob as gb
import cv2
import pickle

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report , confusion_matrix , ConfusionMatrixDisplay,accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization , MaxPooling2D,Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
dataset_path = "/content/archive.zip"

In [ ]:
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(2,2),
#     BatchNormalization(),

#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     BatchNormalization(),

#     Conv2D(128, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     BatchNormalization(),

#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')  # Binary classification
# ])


# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# callbacks = [
#     EarlyStopping(monitor="val_loss", patience=5, verbose=1, restore_best_weights=True)
# ]

# history = model.fit(
#     train_generator  ,
#     validation_data=(test_generator),
#     epochs=10,
#     callbacks=callbacks
# )

# model.save("fracture.h5")
# print("Model saved as fracture.h5")

# with open("fracture.pkl", "wb") as pkl_file:
#     pickle.dump(model, pkl_file)
# print("Model saved as fracture.pkl")


In [ ]:
import zipfile

with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")
print("Archive extracted to /content")

In [ ]:
import os

# Check what's in /content
print("Top-level files/folders after extraction:")
print(os.listdir("/content"))

In [ ]:
dataset_path = "/content/archive (6)"  # or whatever folder name appears
print(os.listdir(dataset_path))

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# used to find all the pathnames matching
import glob as gb
import cv2
import pickle

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report , confusion_matrix , ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization , MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
import os
path_train = r"/content/archive (6)/train"

for folder in os.listdir(path=path_train) :
    file_folder = os.path.join(path_train , folder)
    files = gb.glob(pathname=os.path.join(file_folder , "*.jpg"))
    print(f"Training Data,Found {len(files)} in folder {folder}")


In [ ]:
path_test = r"/content/archive (6)/val"

for folder in os.listdir(path=path_test) :
    file_folder = os.path.join(path_test , folder)
    files = gb.glob(pathname=os.path.join(file_folder , "*.jpg"))
    print(f"Testing Data,Found {len(files)} in folder {folder}")


In [ ]:
def segment_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0) # standard deviation in the X and Y directions
    _, thr = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
                                        #  retrieval mode only retrieves the external contours of objects
    contours, _ = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # reduces the amount of data stored in the contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    return cv2.bitwise_and(image, mask)


In [ ]:
def prepare_image(image, method):
    # Convert to grayscale for methods that require it
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # LBP also applies histogram equalization for better lighting normalization
    if method == 'LBP':
        gray = cv2.equalizeHist(gray)

    return gray


In [ ]:
def feature_extraction_chain_code(image):
    """
    Extract chain code features from the image.
    """
    gray = prepare_image(image, 'chain_code')
    ret, thresh = cv2.threshold(gray, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    chain_code = []

    for contour in contours:
        contour = contour.reshape(-1, 2)
        for i in range(1, len(contour)):
            dx = contour[i][0] - contour[i-1][0]
            dy = contour[i][1] - contour[i-1][1]
            direction = (np.arctan2(dy, dx) + np.pi) // (np.pi / 4)
            chain_code.append(int(direction))

    # Ensure chain code is always 2048-length
    return np.pad(chain_code, (0, 2048 - len(chain_code)), 'constant')[:2048]

def regional_Descriptor(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    inverted_binary = ~binary

    _, thr = cv2.threshold(image, 127, 255, cv2.THRESH_OTSU)

    inverted_binary = ~thr
    contours, _ = cv2.findContours(inverted_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    with_contours=cv2.drawContours(image, contours, -1, (255, 255, 255))

    object=contours[0]
    print('contours= ',object)
    area=cv2.contourArea(object)
    perimeter=cv2.arcLength(object,True)
    compactness=(4*np.pi*area)/(perimeter**2)

    return area, perimeter, compactness

def feature_extraction_glcm(image):
    """
    Extract GLCM texture features from the image.
    """
    gray = prepare_image(image, 'GLCM')
    glcm = texture.greycomatrix(gray, [1], [0], symmetric=True, normed=True)

    contrast = texture.greycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = texture.greycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = texture.greycoprops(glcm, 'homogeneity')[0, 0]

    return np.array([contrast, dissimilarity, homogeneity])

def feature_extraction_lbp(image):
    """
    Extract LBP features from the image.
    """
    gray = prepare_image(image, 'LBP')
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    lbp_histogram, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 59), range=(0, 58))
    lbp_histogram = lbp_histogram.astype('float')
    lbp_histogram /= lbp_histogram.sum()  # Normalize to sum to 1

    # Return first 2048 values or pad
    return lbp_histogram[:2048]

def feature_extraction(image, method='SIFT'):
    """
    Call the appropriate feature extraction method.
    """
    if method == 'chain_code':
        return feature_extraction_chain_code(image)
    elif method == 'regional':
        return regional_Descriptor(image)
    elif method == 'GLCM':
        return feature_extraction_glcm(image)
    elif method == 'LBP':
        return feature_extraction_lbp(image)
    else:
        raise ValueError(f"Unsupported method: {method}")



In [ ]:
labels = {"fractured" : 0 , "not fractured" : 1}


In [ ]:
def load_data(path, target_size=(224, 224)):
    X, y = [], []
    labels = {'fractured': 0, 'not fractured': 1}  # Update based on folder names
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        files = gb.glob(os.path.join(folder_path, "*.jpg"))
        for file in files:
            image = cv2.imread(file)
            if image is not None:  # Check if image is loaded correctly
                image = cv2.resize(image, target_size)  # Resize image
                X.append(image)
                y.append(labels[folder])
    return np.array(X), np.array(y)


In [ ]:
 #Load data
X_train, y_train = load_data(path_train)
X_test, y_test = load_data(path_test)


In [ ]:
print(f"Len X Train Is == {len(X_train)}")
print(f"Len X Test Is == {len(X_test)}")


In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X_train , y_train , test_size=0.2 , random_state=42 , shuffle=True)


print(f"X Train Shape is ==> {X_train.shape}")
print(f"X test Shape is ==> {X_test.shape}")
print(f"y train Shape is ==> {y_train.shape}")
print(f"y test Shape is ==> {y_test.shape}")


In [ ]:
datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = datagen.flow(X_train, y_train, batch_size=32)
test_generator = datagen.flow(X_test, y_test, batch_size=32)


In [ ]:
from tensorflow.keras.optimizers import Adam # Import the Adam optimizer

model = Sequential([
    Flatten(input_shape=(224, 224, 3)),  # Flattening the 2D image into 1D

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy']) # Now Adam is defined

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
)

# Save the model
model.save("fracture_ann.h5")
print("Model saved as fracture_ann.h5")

with open("fracture_ann.pkl", "wb") as pkl_file:
    pickle.dump(model, pkl_file)
print("Model saved as fracture_ann.pkl")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


222/222 ━━━━━━━━━━━━━━━━━━━━ 347s 2s/step - accuracy: 0.5483 - loss: 0.9156 - val_accuracy: 0.6345 - val_loss: 0.6483
Epoch 2/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 335s 2s/step - accuracy: 0.6167 - loss: 0.6912 - val_accuracy: 0.6853 - val_loss: 0.5833
Epoch 3/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 340s 2s/step - accuracy: 0.6678 - loss: 0.6214 - val_accuracy: 0.7236 - val_loss: 0.5321
Epoch 4/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 336s 2s/step - accuracy: 0.7095 - loss: 0.5581 - val_accuracy: 0.7879 - val_loss: 0.4510
Epoch 5/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 336s 2s/step - accuracy: 0.7673 - loss: 0.4938 - val_accuracy: 0.7360 - val_loss: 0.5638
Epoch 6/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: 0.7723 - loss: 0.4701 - val_accuracy: 0.7783 - val_loss: 0.4605
Epoch 7/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 391s 2s/step - accuracy: 0.7900 - loss: 0.4537 - val_accuracy: 0.8483 - val_loss: 0.3585
Epoch 8/30
222/222 ━━━━━━━━━━━━━━━━━━━━ 330s 1s/step - accuracy: 0.8098 - loss: 0.4159 - val_accuracy: 0.833

In [ ]:
loss , acc = model.evaluate(train_generator)
print(f"Loss Training is == > {loss}")
print(f"Accurcy Training is == > {acc}")


In [ ]:
loss , acc = model.evaluate(test_generator)
print(f"Loss Testing is == > {loss}")

print(f"Accurcy Testing is == > {acc}")


In [ ]:
def getlab(n):
    labels = {"fractured" : 0 , "not fractured" : 1} # Define labels within the function scope
    for x, y in labels.items():
        if n == y:
            return x

plt.figure(figsize=(40, 20))
num_images = 6

sample_indices = np.random.randint(0, len(X_train), num_images)

for n, i in enumerate(sample_indices):
    # Read the original image
    img_path = gb.glob(os.path.join(path_train, "*", "*.jpg"))[i]
    img = cv2.imread(img_path)

    # Column 1: Original image
    plt.subplot(num_images, 3, n * 3 + 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"Original: {getlab(y_train[i])}", color="blue", fontsize=18)
    plt.axis('off')

    # Column 2: Segmented image
    segmented_img = segment_image(img)
    plt.subplot(num_images, 3, n * 3 + 2)
    plt.imshow(cv2.cvtColor(segmented_img, cv2.COLOR_BGR2RGB))
    plt.title("Segmented", color="green", fontsize=20)
    plt.axis('off')

    # Column 3: Image with feature extraction
    sift = cv2.SIFT_create()
    keypoints = sift.detect(segmented_img, None)
    img_with_keypoints = cv2.drawKeypoints(segmented_img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.subplot(num_images, 3, n * 3 + 3)
    plt.imshow(cv2.cvtColor(img_with_keypoints, cv2.COLOR_BGR2RGB))
    plt.title("Features", color="red", fontsize=20)
    plt.axis('off')

# Adjust layout for better spacing
plt.tight_layout(pad=3.0)
plt.show()


In [ ]:
pred = model.predict(test_generator)
pred = (pred > 0.5).astype(int)
print(pd.DataFrame(pred[:10] , columns=["Prediction"]))


In [ ]:
cm = confusion_matrix(y_pred=pred , y_true=y_test)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Fractured", "Not Fractured"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
print(classification_report(y_test, pred, target_names=["fractured", "not fractured"]))


In [ ]:
len(X_test)


In [ ]:
num_images = 10

# Randomly select indices             # 1773        10
sample_indices = np.random.randint(0, len(X_test), num_images)

plt.figure(figsize=(15, 10))

for n, i in enumerate(sample_indices):
    actual_img = X_test[i]
    actual_label = y_test[i]

    pred_label = (pred[i] > 0.5).astype(int)

    plt.subplot(num_images, 2, n * 2 + 1)
    plt.imshow(cv2.cvtColor(actual_img, cv2.COLOR_BGR2RGB))
    plt.title(f"Actual: {getlab(actual_label)}", color="blue", fontsize=12)
    plt.axis('off')

    plt.subplot(num_images, 2, n * 2 + 2)
    plt.imshow(cv2.cvtColor(actual_img, cv2.COLOR_BGR2RGB))
    plt.title(f"Predicted: {getlab(pred_label)}", color="red", fontsize=12)
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# prompt: bro make a ui to predict

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load your trained model
model = load_model("fracture_ann.h5")

# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array

# Function to predict the class
def predict_fracture(img_path):
    processed_image = preprocess_image(img_path)
    prediction = model.predict(processed_image)
    if prediction[0][0] > 0.5:
        return "Not Fractured"
    else:
        return "Fractured"


# UI elements
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
output_label = widgets.Label()

def on_upload_change(change):
    if upload_button.value:
        uploaded_file = list(upload_button.value.values())[0]
        file_name = list(upload_button.value.keys())[0]

        with open(file_name, 'wb') as f:
          f.write(uploaded_file['content'])

        prediction_result = predict_fracture(file_name)
        output_label.value = f"Prediction: {prediction_result}"
        clear_output(wait=True)
        display(upload_button, output_label)

upload_button.observe(on_upload_change, names='_counter')

# Display the UI
display(upload_button, output_label)


In [ ]:
import numpy as np
!pip install gradio --upgrade

import gradio as gr

# Assuming your model is loaded and ready to use as 'model'

def predict_image(image):
  # Preprocess the image (resize, normalize, etc.) as needed for your model
  image = cv2.resize(image, (224, 224)) # Resize the image to (224, 224) here
  image = image / 255.0
  image = np.expand_dims(image, axis=0)  # Add batch dimension

  prediction = model.predict(image)
  predicted_label = (prediction > 0.5).astype(int)[0][0]

  return "Fractured" if predicted_label == 1 else "Not Fractured"

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(),  # Remove the 'shape' argument
    outputs="text",
    title="Bone Fracture Detection",
    description="Upload an X-ray image to predict if it shows a fracture.",

)

iface.launch()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.utils import plot_model

# # Load the trained model
# model = tf.keras.models.load_model("fracture.h5")

# # Print model summary in text format
# model.summary()

# # Save and display model architecture as an image
# plot_model(model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True)

# # If using Jupyter Notebook, display the image
# from IPython.display import Image
# Image(filename="model_architecture.png")


In [ ]:
from tensorflow.keras.utils import plot_model # Import the plot_model function
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam


# Define ANN model
model = Sequential([
    Flatten(input_shape=(224, 224, 3)),  # Flattening the 2D image into 1D

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Visualize model architecture
plot_model(model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True) # Now the model has layers defined